In [7]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path: sys.path.append(module_path)

import pandas as pd
import json
from tqdm import tqdm
import pickle
from pathlib import Path

from RMALoaders import *

### Helper Functions

In [8]:
def mkdir(path):
    if not os.path.exists(path): os.makedirs(path)
    return path

In [11]:
desktop_path = Path.home() / 'Desktop'
connectivity_path = desktop_path / 'data' / 'connectivity'
print(connectivity_path)

/home/ikharitonov/Desktop/data/connectivity


### Main Parameters

In [12]:
main_structure = 'VISpm'
main_structure_object = RMAStructure(acronym=main_structure)
main_structure_id = main_structure_object.id

HEMISPHERE_TO_FILTER = 1 # only experiments injected in this hemisphere will be selected

projection_metric = 'normalized_projection_volume' # for later export into brainrender

INJECTION_VOLUME_THRESHOLD = 0.5

read_filtered_area_experiment_id_dict_2 = True # read area-experiment_id dictionary from existing file
load_unionized_data = True # load the unionized data from experiments specified in dictionary above

### Loading csv file with metadata of experiments projecting to the structure of interest

In [13]:
filename = main_structure + ".csv"
df = pd.read_csv(connectivity_path / 'connectivity_target_experiment_lists' / filename)
df

,id,transgenic-line,product-id,structure-id,structure-abbrev,structure-name,name,injection-volume,injection-structures,gender,strain,sum,structure-color,num-voxels,injection-coordinates,selected,experiment_page_url
0,100141599,NaN,5,394,VISam,Anteromedial visual area,378-757,0.125483,"[{""id""=>394, ""abbreviation""=>""VISam"", ""name""=>...",M,C57BL/6J,4.180800e-01,08858c,NaN,"[7900, 580, 7380]",False,http://connectivity.brain-map.org/projection/e...
1,479673174,Emx1-IRES-Cre,35,409,VISl,Lateral visual area,Emx1-IRES-Cre-191135,0.605379,"[{""id""=>385, ""abbreviation""=>""VISp"", ""name""=>""...",M,B6.129,3.220639e-01,08858c,NaN,"[9140, 2290, 2390]",False,http://connectivity.brain-map.org/projection/e...
2,512315551,Emx1-IRES-Cre,36,385,VISp,Primary visual area,Emx1-IRES-Cre-234272,1.614223,"[{""id""=>385, ""abbreviation""=>""VISp"", ""name""=>""...",M,NaN,3.149679e-01,08858c,NaN,"[8460, 1630, 2340]",False,http://connectivity.brain-map.org/projection/e...
3,309004492,NaN,5,385,VISp,Primary visual area,C57BL/6-155459,1.083030,"[{""id""=>385, ""abbreviation""=>""VISp"", ""name""=>""...",M,C57BL/6J,2.642719e-01,08858c,NaN,"[9450, 2180, 8700]",False,http://connectivity.brain-map.org/projection/e...
4,180296424,NaN,5,385,VISp,Primary visual area,378-1815,0.814006,"[{""id""=>385, ""abbreviation""=>""VISp"", ""name""=>""...",M,C57BL/6J,2.524630e-01,08858c,NaN,"[9290, 2220, 9410]",False,http://connectivity.brain-map.org/projection/e...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2790,122641784,NaN,5,382,CA1,Field CA1,378-1250,0.069547,"[{""id""=>382, ""abbreviation""=>""CA1"", ""name""=>""F...",M,C57BL/6J,5.745112e-11,7ed04b,NaN,"[8040, 5550, 8970]",False,http://connectivity.brain-map.org/projection/e...
2791,126352037,NaN,5,128,MRN,Midbrain reticular nucleus,378-1281,0.018755,"[{""id""=>128, ""abbreviation""=>""MRN"", ""name""=>""M...",M,C57BL/6J,5.124817e-11,ff90ff,NaN,"[9860, 3760, 6580]",False,http://connectivity.brain-map.org/projection/e...
2792,168401109,Pvalb-IRES-Cre,5,48,ACAv,"Anterior cingulate area, ventral part",Pvalb-IRES-Cre-5045,0.194367,"[{""id""=>39, ""abbreviation""=>""ACAd"", ""name""=>""A...",M,B6.129,4.817813e-11,40a666,NaN,"[5100, 1750, 6010]",False,http://connectivity.brain-map.org/projection/e...
2793,168302154,Grik4-Cre,5,647,COAp,"Cortical amygdalar area, posterior part",Grik4-Cre-5027,0.325879,"[{""id""=>319, ""abbreviation""=>""BMA"", ""name""=>""B...",M,C57BL/6J,3.168943e-11,61e7b7,NaN,"[7200, 6540, 8440]",False,http://connectivity.brain-map.org/projection/e...


In [14]:
# https://allensdk.readthedocs.io/en/latest/_modules/allensdk/api/queries/mouse_connectivity_api.html#MouseConnectivityApi.get_structure_unionizes
# using this approach, unionized data would have to be downloaded for every experiment (might take a lot of time e.g. 1855 experiments for RSPagl)
RMAUnionizedData(experiment_id=512315551, select_structure_id=894).data

,hemisphere_id,id,is_injection,max_voxel_density,max_voxel_x,max_voxel_y,max_voxel_z,normalized_projection_volume,projection_density,projection_energy,projection_intensity,projection_volume,section_data_set_id,structure_id,sum_pixel_intensity,sum_pixels,sum_projection_pixel_intensity,sum_projection_pixels,volume,structure
0,2,640689589,False,0.696321,6920,350,6900,0.013240,0.016449,9.820845,597.049358,0.021373,512315551,894,1.216443e+11,1.060688e+09,1.041685e+10,1.744722e+07,1.299343,"{'acronym': 'RSPagl', 'atlas_id': 394, 'color_..."
1,1,640690930,False,1.000000,9720,1630,3410,0.068989,0.099411,206.254527,2074.762553,0.111363,512315551,894,3.327241e+11,9.144732e+08,1.886142e+11,9.090883e+07,1.120230,"{'acronym': 'RSPagl', 'atlas_id': 394, 'color_..."
2,3,640692612,False,1.000000,9720,1630,3410,0.082229,0.054859,100.767021,1836.824888,0.132736,512315551,894,4.543683e+11,1.975161e+09,1.990311e+11,1.083561e+08,2.419572,"{'acronym': 'RSPagl', 'atlas_id': 394, 'color_..."


### Get the reference list of brain areas

In [15]:
struct_set_id = 167587189 # Curated list of non-overlapping substructures at a mid-ontology level

structure_sets = RMAStructureSet()
struct_set = structure_sets.get_structure_set(id=struct_set_id)
struct_set

,acronym,atlas_id,color_hex_triplet,depth,failed,failed_facet,graph_id,graph_order,hemisphere_id,id,...,neuro_name_structure_id,neuro_name_structure_id_path,ontology_id,parent_structure_id,safe_name,sphinx_id,st_level,structure_id_path,structure_name_facet,weight
0,PAG,240.0,FF90FF,5,False,734881840,1,838,3,795,...,None,None,1,323,Periaqueductal gray,839,8,/997/8/343/313/323/795/,3260726339,8690
1,ARH,27.0,FF5D50,6,False,734881840,1,733,3,223,...,None,None,1,157,Arcuate hypothalamic nucleus,734,8,/997/8/343/1129/1097/157/223/,218062747,8690
2,ORBm,232.0,248A5E,7,False,734881840,1,264,3,731,...,None,None,1,714,Orbital area medial part,265,9,/997/8/567/688/695/315/714/731/,3012751712,8690
3,LSv,174.0,90CBED,7,False,734881840,1,589,3,266,...,None,None,1,242,Lateral septal nucleus ventral part,590,9,/997/8/567/623/477/275/242/266/,1660459064,8690
4,PD,255.0,FF5547,6,False,734881840,1,746,3,914,...,None,None,1,141,Posterodorsal preoptic nucleus,747,8,/997/8/343/1129/1097/141/914/,2759126254,8690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,SSp-bfd,748.0,188064,8,False,734881840,1,51,3,329,...,None,None,1,322,Primary somatosensory area barrel field,52,9,/997/8/567/688/695/315/453/322/329/,3406319794,8690
312,OT,235.0,80CDF8,6,False,734881840,1,577,3,754,...,None,None,1,493,Olfactory tubercle,578,8,/997/8/567/623/477/493/754/,1598442672,8690
313,SubG,464.0,FF909F,7,False,734881840,1,710,3,321,...,None,None,1,1014,Subgeniculate nucleus,711,8,/997/8/343/1129/549/856/1014/321/,3545734096,8690
314,SNr,330.0,FF90FF,5,False,734881840,1,822,3,381,...,None,None,1,323,Substantia nigra reticular part,823,8,/997/8/343/313/323/381/,1375238552,8690


### Check that all projecting experiments are linked to an area in the list

In [16]:
# SANITY CHECK

# make a copy of experiment df
unmatched_df = df.copy()
num_collected_exps = 0

# loop through brain areas, removing experiments successfully matching with an area
for area_id in struct_set['id']:
    indexes_to_drop = df.index[df['structure-id']==area_id]
    num_collected_exps += len(indexes_to_drop)
    unmatched_df = unmatched_df.drop(indexes_to_drop)
    # if area_id == 329: break

# display df with unmatched experiments
print(num_collected_exps)
unmatched_df

2795


,id,transgenic-line,product-id,structure-id,structure-abbrev,structure-name,name,injection-volume,injection-structures,gender,strain,sum,structure-color,num-voxels,injection-coordinates,selected,experiment_page_url


dataframe is empty

167587189 -> sanity checked -> all experiments, from corresponding metadata downloaded in csv in connectivity_target_experiment_lists, are assigned to an area -> no missed experiments due to injection area (labelling convention) mismatch

### Collect experiments in {area_id: [exp_id_1, exp_id_2, ...]} dictionary

In [17]:
# Number of experiments from injections across all brain areas targeting RSPagl
total_num_collected_exps = 0
area_experiment_id_dict = {}

for area_id in struct_set['id']:
    total_num_collected_exps += len(df[df['structure-id']==area_id])
    area_experiment_id_dict[area_id] = list(df[df['structure-id']==area_id]['id'])
    # print("Experiments in area",struct_set[struct_set['id']==area_id]['acronym'].item(),"=", len(df[df['structure-id']==area_id]))
print(total_num_collected_exps,"experiments collected in the dictionary")
print(len(area_experiment_id_dict),"number of areas collected in the dictionary")
print("area",795)
print(area_experiment_id_dict[795])
print("area",223)
print(area_experiment_id_dict[223])

2795 experiments collected in the dictionary
316 number of areas collected in the dictionary
area 795
[300076066, 496114558, 267029447, 266500714, 272829745, 287247978, 272699357, 266099165, 182144176, 300166697, 287712779, 156979283, 158376179, 120761491, 182280207, 160538548, 302053755, 301540850, 120571672, 262188772, 162020630, 298079928, 147635309, 301671287, 128002057, 304949216, 300111793, 113096571, 180524412, 267030155, 299856390, 543880631]
area 223
[263369222, 171482142, 175738378, 286726777, 176431817, 241278553, 146554676, 232311236, 178282527, 181891892, 158142090, 286318327, 586447435, 146660999, 232310521, 298105299, 159751184]


### Filter out experiments where target structure and injection structures overlap

In [18]:
# Filtering out experiments with overlapping injection and target structures

filtered_area_experiment_id_dict_1 = {}
exps_removed = []

for area_id, exps in area_experiment_id_dict.items():
    # if a V2m structure id is contained in injection-structures of an experiment, drop that experiment (id) from area_experiment_id_dict
    exps_ids_to_retain = []
    for exp_id in exps:
        # Loading and formatting dictionary with experiment's injection structures from Allen metadata
        inj_structs_dict = json.loads(df[df['id']==exp_id]['injection-structures'].item().replace("=>",":"))
        # Getting ids of experiment's injection structures
        inj_structs_id_list = [x['id'] for x in inj_structs_dict]
        if main_structure_id in inj_structs_id_list: exps_removed.append(exp_id)
        else: exps_ids_to_retain.append(exp_id)
    filtered_area_experiment_id_dict_1[area_id] = exps_ids_to_retain
print(len(exps_removed),"experiments removed:")
print(exps_removed)

98 experiments removed:
[496113850, 531443949, 599076623, 606778738, 113846682, 576341623, 646525997, 593018150, 576036240, 495345959, 595884140, 554333581, 577298618, 127991964, 114474520, 584511827, 288168426, 657042668, 598605738, 517962765, 656632388, 597256577, 294484177, 495877413, 268040381, 297597186, 482640524, 511817919, 168097187, 579628905, 297627858, 646527844, 584511119, 100141599, 159753308, 571100135, 518742338, 297233422, 184167484, 552431726, 524667618, 294434867, 288264047, 517326050, 297670312, 518619451, 559878074, 516491813, 606930364, 523714940, 268038969, 478678606, 651703553, 293821389, 653191449, 597007143, 557347149, 586041882, 552279683, 520012330, 535696750, 482581199, 520018181, 175018829, 267493760, 272873704, 146078721, 294481346, 512314723, 146077302, 572588941, 501006221, 495345251, 519186737, 485237081, 590987294, 257667830, 502074651, 182896517, 299733445, 523718823, 561506791, 297946154, 570460301, 557342452, 298324391, 636803957, 301618122, 5021809

### Checking the hemisphere of injection for each experiment

#### average template dimensions (from https://www.sciencedirect.com/science/article/pii/S0092867420304025)

<img src="https://global.discourse-cdn.com/standard10/uploads/brainobservatory/original/1X/44f3499fd49d9396d9d12597725afd41693582f5.png" alt="coordinate_system" width=900 />

13.2 mm x 8.0 mm x 11.4 mm

13200 µm x 8000 µm x 11400 µm

In [19]:
def check_hemisphere(experiment_id, structure_id):
    """
    Returns the hemisphere_id for a given injection structure and experiment id. If the injection spans both hemispheres, the one with higher volume is chosen.
    
        Parameters:
            experiment_id (int): id of the experiment (section_data_set_id).
            structure_id (int): id of the injection structure.
            
        Returns:
            hemisphere_id (int): 1 for left hemisphere and 2 for right hemisphere. If there is no injection structure with specified structure_id, 0 is returned.
    """
    temp_data = RMAUnionizedData(experiment_id=experiment_id).data
    temp_data = temp_data[temp_data['structure_id']==structure_id].reset_index(drop=True)

    z_coord = temp_data['max_voxel_z'].unique()
    # if there is data in both hemispheres, choose the one with higher volume
    if len(z_coord) > 1:
        z_coord = [temp_data.iloc[temp_data['volume'].idxmax()]['max_voxel_z']]

    if len(z_coord)==0: return 0
    elif z_coord[0] < 5700: return 1
    elif z_coord[0] >= 5700: return 2

In [20]:
check_hemisphere(512315551, 894)

1

### Removing experiments which have not been injected into specified hemisphere

In [21]:
# Removing all experiments that were not injected in the specified hemisphere

if read_filtered_area_experiment_id_dict_2:
    # Reading from a file
    filename = 'filtered_area_experiment_id_dict_2_hem_'+str(HEMISPHERE_TO_FILTER)+'_'+main_structure+'_vol.pkl'
    with open(connectivity_path / filename, 'rb') as f: filtered_area_experiment_id_dict_2 = pickle.load(f)
else:
    # Copy the dictionary
    filtered_area_experiment_id_dict_2 = {k:v.copy() for k,v in filtered_area_experiment_id_dict_1.items()}
    exps_removed = []
    for area, exps in tqdm(filtered_area_experiment_id_dict_2.items()):
        for e in exps:
            ind = exps.index(e)
            hem = check_hemisphere(e, area)
            if hem != HEMISPHERE_TO_FILTER: exps_removed.append(filtered_area_experiment_id_dict_2[area].pop(ind))
    print(len(exps_removed),"experiments removed:")
    print(exps_removed)
    # Saving to a file
    filename = 'filtered_area_experiment_id_dict_2_hem_'+str(HEMISPHERE_TO_FILTER)+'_'+main_structure+'_vol.pkl'
    with open(connectivity_path / filename, 'wb') as f: pickle.dump(filtered_area_experiment_id_dict_2, f)

In [22]:
"""
Output of cell above for RSPagl hem 1:
1235 experiments removed:
[300076066, 267029447, 272829745, 266099165, 302053755, 287712779, 301540850, 160538548, 147635309, 300111793, 120571672, 298079928, 304949216, 299856390, 301671287, 543880631, 292624169, 263369222, 175738378, 241278553, 286726777, 181891892, 286318327, 146660999, 586447435, 159751184, 126860974, 292477301, 277956496, 265820216, 303537993, 182887258, 304997333, 177460028, 286417464, 299828473, 605660419, 287028480, 518745840, 127139568, 558580065, 278435864, 626107114, 585026021, 300078194, 287770700, 299403823, 267999034, 640282128, 527393818, 585775993, 181859467, 167793416, 293750063, 299447153, 288321385, 302014692, 112672974, 540685246, 581641279, 176432524, 287044088, 183459175, 127470976, 179902786, 292211743, 287459601, 176900059, 112827164, 178488152, 147790922, 587294457, 310176384, 287808449, 287446625, 292960052, 127468854, 171065200, 171064488, 301421253, 127085005, 114248377, 265135682, 112789899, 143515102, 182029881, 126710034, 125437921, 301875208, 126843905, 127041832, 286609510, 121145750, 302217570, 146984915, 509602066, 302084009, 113696423, 187269162, 300641829, 288171256, 120916102, 286834976, 297855879, 287951098, 177783918, 156395997, 257636467, 287996596, 297945448, 520728084, 117298988, 296047084, 156743264, 286314623, 120494024, 182616478, 166082842, 127084296, 156786234, 591612976, 287461719, 477836675, 166461193, 597007858, 277957908, 179640955, 179641666, 272697944, 159651060, 584903636, 114250546, 307295727, 297593942, 116903968, 301617370, 146858755, 557200148, 297225422, 266487079, 297629988, 301210923, 530018580, 530008580, 166461899, 294266031, 283017324, 286835688, 287036898, 309564818, 181057754, 175106053, 523705737, 267928135, 305677409, 180674463, 126116142, 127649713, 159375743, 300167479, 267928844, 175740500, 268076421, 168300739, 272830456, 267607635, 183058837, 294356922, 272875132, 168002780, 183329222, 293365328, 286646170, 284665639, 268399145, 168301446, 272875838, 301324895, 298001595, 264320076, 304948510, 264248605, 113037759, 267397226, 267609756, 292321278, 175818392, 292478008, 286553311, 298232793, 287171983, 305320171, 114009636, 264872385, 272970039, 299857813, 293115527, 113780276, 100148143, 264709761, 287097454, 182029174, 112672268, 114399224, 293011049, 305400137, 278504263, 293011759, 301122593, 112167395, 159996191, 175142304, 127042540, 127349111, 178283952, 171409520, 100140756, 286299886, 298404154, 298182842, 586052938, 281459203, 287027773, 516846757, 310437091, 298004028, 263241470, 298003295, 278070717, 642180077, 266248776, 287458895, 293914766, 514513838, 293787288, 293942897, 166054929, 100140949, 249396394, 521264566, 166271142, 538078619, 516838033, 166325321, 159097209, 586042591, 267658040, 157711043, 298000880, 168363168, 272735744, 288264753, 292172100, 308721884, 526502961, 292124058, 308027576, 167902586, 287769286, 112423392, 127560043, 266564027, 183618845, 293433283, 288263341, 300843826, 300236056, 485903475, 187268452, 513826657, 175372863, 113442864, 180436360, 292372636, 174361746, 304693450, 300623424, 156314054, 157063781, 293701770, 477271169, 266913240, 267749821, 120875816, 249327301, 180075597, 306268688, 171276330, 177322838, 168096467, 301583889, 512130913, 123664417, 125801739, 265930674, 147706228, 158838128, 515418047, 300691325, 272826010, 272918345, 161178152, 267760731, 120812686, 266249483, 112229814, 286312782, 292792016, 148964212, 303615412, 286300594, 298352336, 515254107, 304617742, 178283239, 518015408, 182336846, 160296448, 113314337, 179642401, 175728165, 146859480, 166156241, 264631432, 146921849, 304612686, 171021122, 182294687, 299623085, 267150949, 168302154, 266962653, 301618828, 178382220, 266488504, 179644545, 249402769, 177780284, 179902073, 182615063, 182615771, 301120618, 301326316, 112308468, 180523704, 114429338, 278318653, 554022330, 181890477, 112459547, 304970618, 265945645, 241279261, 127761449, 146795148, 278179088, 120281646, 157766259, 309385637, 258315443, 125830911, 265946352, 267810394, 168094300, 272917631, 301672755, 142655513, 262215150, 277853501, 120874405, 126115436, 181180790, 180601025, 162019589, 272829024, 520619072, 157952778, 100141596, 287714197, 287448038, 299761293, 116905391, 113368085, 183375545, 294173584, 287029186, 147354951, 113846682, 299759175, 127083591, 266646036, 265293391, 305426208, 525796603, 267538006, 278398949, 157765542, 126188607, 127041126, 147790181, 305293928, 127353220, 272738620, 160151570, 292794673, 115958825, 180073473, 304473503, 305024724, 232311959, 142654100, 642967852, 182794184, 301264977, 287952517, 146553971, 127397469, 183225124, 516278420, 182226133, 286837316, 301265683, 287769992, 518207061, 300319630, 305973112, 278317239, 181891184, 299857096, 298798846, 301988879, 159602279, 303536573, 264564375, 552280478, 268206050, 156202979, 180628971, 312240825, 159331462, 266839077, 176897793, 167373923, 171067319, 305125123, 121145045, 478818791, 157654817, 299447873, 278317945, 299829181, 159750477, 303785454, 306959448, 272928602, 177892379, 292318449, 159651770, 297985755, 168229820, 112936582, 127991964, 174583904, 273065264, 147354242, 174583187, 113400134, 581328778, 139311530, 258914806, 304761539, 156978574, 121509005, 287950390, 267611175, 167571459, 266174045, 156393801, 168095041, 114755811, 182465608, 303474463, 287879384, 301324157, 168230532, 181116850, 180296424, 307558646, 309113907, 113887162, 309004492, 307296433, 307137980, 307743253, 307321674, 307297141, 296052133, 307557934, 266250195, 309003780, 100141219, 300929973, 299859225, 294482052, 304586645, 268040381, 114008926, 288171964, 166326736, 304565427, 277714322, 277616630, 121510421, 184074388, 297629282, 301616660, 277712166, 183173527, 272821309, 623855077, 293472335, 605657236, 257667118, 656842599, 181258571, 159550125, 477475894, 511234217, 308385516, 156545918, 304762965, 299781566, 267931685, 671033786, 266100645, 166269819, 304564721, 304585910, 166562678, 288261928, 177781745, 670657874, 287224335, 626118851, 167511639, 292793967, 648051857, 286773358, 182184486, 266836749, 176882966, 176901480, 294312537, 477269754, 638313933, 616677276, 638978767, 156784823, 596828910, 298350212, 596571282, 575772121, 157767683, 168515938, 656657344, 638314843, 648048922, 623438145, 146986331, 564688610, 184075807, 127711098, 120280939, 100148554, 204832205, 538833505, 265947058, 170784358, 294040662, 113504763, 147134401, 287713485, 146012184, 100141599, 100148503, 159753308, 184167484, 267657327, 297233422, 297670312, 294434867, 288264047, 156671933, 516491813, 293821389, 204908781, 159510205, 539010002, 310377782, 165035106, 298048079, 515410820, 113506174, 298048787, 114046440, 171020416, 158373181, 278508779, 112372418, 287667137, 293254286, 117302771, 113225519, 113369603, 176887774, 158738894, 119847544, 506426778, 288322091, 127795906, 299446445, 550155867, 298795868, 152994878, 556343427, 527051458, 113368898, 640285199, 549809266, 516529585, 294314037, 293009970, 286556914, 160152987, 167213641, 177903648, 309794438, 292622743, 263553934, 287668550, 112881858, 113936696, 267106046, 159945113, 300622004, 287539649, 180916954, 182793477, 287995889, 166055636, 265292679, 278258073, 272822110, 168162771, 100141454, 176433237, 294532700, 477835774, 267930978, 292034715, 298000163, 277957202, 267813224, 585025284, 292374068, 265929196, 263242463, 120814821, 181598954, 305487964, 298718072, 157710335, 606785720, 177459319, 587659400, 292172846, 166460484, 156492394, 168454779, 266645328, 292210312, 159372889, 180719293, 477037203, 182795499, 183617432, 614735393, 258915564, 287807743, 157826227, 127908879, 141602484, 166082128, 297669605, 158139883, 297987980, 287807030, 569264117, 293827470, 584513749, 297946935, 596798450, 156394513, 288169842, 168002073, 297947641, 159373612, 300235349, 292208876, 294533406, 267761438, 278175580, 478782005, 168165712, 113884251, 126653015, 157556400, 298272589, 294399325, 298274021, 159511623, 278179794, 298720191, 182467736, 100141796, 286554724, 292791310, 159320367, 127091083, 156741826, 288324211, 180709230, 278171908, 170721670, 168164972, 183471174, 303783725, 278174451, 293889216, 307766627, 268321221, 300843120, 126841788, 299404532, 549362997, 305026861, 298835152, 146045723, 273026584, 299245589, 125436508, 302739608, 304720034, 126710740, 148197327, 273055501, 558673113, 287494320, 278433737, 266582782, 161463521, 266816894, 292374777, 138059031, 511971714, 301016175, 313141786, 292790567, 545427588, 183901489, 268415561, 552284594, 183284388, 264565965, 539028976, 294356216, 565721603, 311845972, 167442042, 300923916, 304998039, 162018169, 168005102, 167117360, 286319739, 292961470, 515191874, 301875966, 170860801, 293469501, 549361039, 272819994, 287666431, 299896150, 182041643, 175072921, 180981417, 305645132, 125361005, 287246555, 182460343, 587060515, 127909584, 294355509, 114472860, 183071513, 146858006, 182090318, 100149109, 184159706, 116903230, 303582161, 182686935, 305488671, 266963362, 165975810, 157954927, 293820681, 183105218, 301877386, 156202272, 113144533, 277710753, 299244730, 168515225, 272700063, 309701202, 158435826, 303709219, 292960764, 143512399, 121146455, 278261300, 126522350, 299783689, 296048512, 313327028, 262536037, 184168899, 264320859, 112596790, 168663472, 264697714, 287492899, 170858675, 126190743, 306270474, 170263370, 304858700, 305321883, 113095845, 511549277, 112826458, 147162736, 586054741, 287880821, 313325371, 158258062, 286556208, 182337561, 308641549, 176886958, 286319033, 114290225, 266489212, 277854916, 574572418, 127798146, 127867804, 127651139, 160540751, 127796728, 301062306, 156314762, 166054222, 125831616, 159021559, 265633461, 267153115, 302085421, 113553300, 293942188, 168616111, 303710632, 183057424, 182888003, 305269070, 183329991, 299996344, 113935990, 146012934, 267813932, 508775240, 167439900, 146658879, 183282261, 183282970, 292319865, 293914056, 301466249, 128056535, 129567943, 293252166, 298079222, 165974379, 303616127, 272929308, 286301303, 521402511, 293548317, 277851379, 297712045, 168615344, 146470726, 287533790, 293008559, 175373569, 300078901, 287993060, 127255254, 167904966, 287665706, 299653551, 141601779, 170949055, 267103498, 287715650, 264874516, 114755099, 159994059, 158738180, 298049545, 113554719, 294005186, 277800288, 182805258, 272873704, 527808181, 160294327, 265943460, 158373958, 300318924, 158915602, 120437703, 297231636, 166459778, 267999740, 174781014, 100147785, 182515576, 301674988, 292623457, 301057735, 510124187, 157063074, 170859382, 273025872, 180707817, 267929554, 112935169, 114292355, 297652799, 266644610, 182803137, 584902900, 178284661, 166566678, 278257366, 266643194, 166323186, 157768393, 266964788, 146078721, 278513131, 126523066, 523178542, 167118084, 126646502, 278511717, 175158132, 176886238, 267764292, 194948535, 265138021, 181786681, 147049515, 298835859, 304760810, 167026321, 146747721, 241280698, 159433187, 268163228, 273025166, 267958444, 263976175, 557972433, 286608092, 304996627, 122641078, 158321996, 308549214, 304721447, 286647583, 278066728, 177905562, 161460153, 113766744, 301800314, 513773998, 267762859, 514506712, 158314278, 139519496, 156493815, 477834984, 160080068, 520336173, 520342605, 157952068, 146659588, 267398651, 263780018, 298178912, 264096244, 586377476, 287460307, 146856593, 146593590, 125833030, 139426984, 112458114, 114428632, 287599685, 181858761, 592391705, 177783204, 183472596, 293432575, 286646877, 298758841, 166323896, 286774770, 278069982, 267705378, 288262635, 165644051, 267101850, 266964075, 181179335, 159551564, 516848198, 293431869, 183104511, 286728190, 158374671, 146984209, 300168916, 112307046, 299897573, 171485060, 506117918, 299655676, 485875903, 310436191, 551350026, 175374275, 127710392, 292480129, 555011865, 267213793, 268208632, 112425523, 266837456, 100141434, 300927483, 272827141, 485847695, 159433905, 286775476, 120811946, 300624130, 180404418, 292375491, 581350498, 519164644, 127089669, 298178204, 287953929, 157062358, 166083557, 293470216, 184075100, 176429574, 159321806, 294516943, 170261951, 158141324, 298830868, 294481346, 182896517, 257667830, 298324391, 127650431, 180435652, 180403712, 268389532, 304537794, 168664192, 120493315, 168361750, 294175704, 516526160, 286610923, 180405830, 181093729, 286610216, 286554017, 301119459, 182226839, 297894255, 171019004, 299656382, 292479421, 301130077, 181777177, 305380411, 278503555, 267608343, 113784293, 175817683, 113554008, 298833033, 267959197, 300642574, 266693274, 272968624, 182280916, 159648854, 180525136, 268323342, 302050617, 555739999, 286482701, 265712971, 161458737, 139520203, 265288825, 125801033, 585911240, 265291552, 303783015, 159319654, 299829892, 268038262, 299782983, 301179679, 478780588, 287714903, 298797288, 168401109, 266412788, 302079862, 156255078, 299732033, 292531359, 112424102, 126907302, 297654263, 112951804, 175819113, 127866392, 176881134, 100141473, 297671018, 166533924, 485022109, 294435580, 288170549, 266486371, 183374804, 266172624, 159602992, 177907797, 298718778, 297628576, 297668898, 511234957, 277801701, 265820951, 286772650, 178487444, 167441329, 159223769, 263370720, 304335875, 167904255, 158914182, 478096249, 303534443, 299895444, 513775257, 307910595, 120762196, 180982124, 514505957, 160537018, 293366741, 292620968, 117317884, 160537796, 112458831, 146553266, 113766038, 272697238, 160540013, 183009881, 159223001, 158020947, 287995180, 120570964, 175732996, 126853068, 175072215, 278434443, 293366035, 477924853, 292959343, 287994474, 158019342, 310175667, 127224133, 156670520, 575683020, 114399934]
"""

'\nOutput of cell above for RSPagl hem 1:\n1235 experiments removed:\n[300076066, 267029447, 272829745, 266099165, 302053755, 287712779, 301540850, 160538548, 147635309, 300111793, 120571672, 298079928, 304949216, 299856390, 301671287, 543880631, 292624169, 263369222, 175738378, 241278553, 286726777, 181891892, 286318327, 146660999, 586447435, 159751184, 126860974, 292477301, 277956496, 265820216, 303537993, 182887258, 304997333, 177460028, 286417464, 299828473, 605660419, 287028480, 518745840, 127139568, 558580065, 278435864, 626107114, 585026021, 300078194, 287770700, 299403823, 267999034, 640282128, 527393818, 585775993, 181859467, 167793416, 293750063, 299447153, 288321385, 302014692, 112672974, 540685246, 581641279, 176432524, 287044088, 183459175, 127470976, 179902786, 292211743, 287459601, 176900059, 112827164, 178488152, 147790922, 587294457, 310176384, 287808449, 287446625, 292960052, 127468854, 171065200, 171064488, 301421253, 127085005, 114248377, 265135682, 112789899, 14351

In [23]:
df[df['id']==300076066]

,id,transgenic-line,product-id,structure-id,structure-abbrev,structure-name,name,injection-volume,injection-structures,gender,strain,sum,structure-color,num-voxels,injection-coordinates,selected,experiment_page_url
984,300076066,Dbh-Cre_KH212,5,795,PAG,Periaqueductal gray,Dbh-Cre_KH212-125311,0.013064,"[{""id""=>162, ""abbreviation""=>""LDT"", ""name""=>""L...",F,C57BL/6J,0.000638,ff90ff,NaN,"[9940, 3700, 6280]",False,http://connectivity.brain-map.org/projection/e...


In [24]:
check_hemisphere(300076066,795)

2

In [25]:
# query_structure_path_acronyms_by_acronym('LDT')
RMAStructure(acronym='LDT').get_structure_path()

'root/grey/BS/HB/P/P-sat/LDT'

In [26]:
print(filtered_area_experiment_id_dict_2.keys())

dict_keys([795, 223, 731, 266, 914, 207, 1049, 609, 35, 425, 1009, 926, 38, 1105, 59, 302, 390, 591, 262, 872, 101, 1061, 287, 1048, 1084, 30, 564, 604, 238, 619, 689, 378, 985, 409, 88, 362, 880, 614, 422, 255, 576073699, 726, 978, 549009223, 333, 27, 184, 361, 131, 170, 886, 746, 483, 583, 23, 706, 677, 1037, 133, 484682470, 966, 353, 1025, 1093, 151, 763, 280, 369, 830, 765, 507, 231, 356, 839, 549009215, 647, 463, 536, 206, 612, 169, 258, 898, 757, 66, 968, 484682508, 182305689, 621, 210, 4, 852, 177, 374, 215, 589508451, 639, 263, 982, 178, 534, 136, 75, 894, 607344830, 106, 975, 437, 222, 970, 1027, 347, 1031, 918, 1077, 773, 733, 15, 12, 574, 549009219, 10671, 345, 128, 581, 1033, 127, 1113, 589508447, 7, 100, 1017, 385, 470, 843, 181, 203, 920, 394, 149, 162, 350, 358, 194, 1041, 502, 998, 96, 72, 235, 576073704, 398, 560581559, 1018, 246, 1007, 599626927, 147, 993, 118, 541, 549009227, 576, 312782546, 1044, 115, 629, 318, 298, 566, 58, 44, 879, 312782574, 271, 723, 922, 1039, 

In [27]:
print(len(filtered_area_experiment_id_dict_2))

316


### Save/load unionized data in csv files for every experiment

In [28]:
area_experiment_unionized_data = {}
if load_unionized_data:
    # each csv file with unionized data is read into memory and rows (data on all hemispheres) corresponding to the target structure are saved into a dictionary
    foldername = 'unionized_data_from_hem_'+str(HEMISPHERE_TO_FILTER)+'_'+main_structure+'_projecting_experiments'
    for area, experiments in tqdm(filtered_area_experiment_id_dict_2.items()):
        area_experiment_unionized_data[area] = {}
        for e in experiments:
            filename = 'area_id_'+str(area)+'exp_id_'+str(e)+'.csv'
            temp_df = pd.read_csv(connectivity_path / foldername / filename)
            area_experiment_unionized_data[area][e] = temp_df
else:
    # for each area, each experiment, download unionized data
    foldername = 'unionized_data_from_hem_'+str(HEMISPHERE_TO_FILTER)+'_'+main_structure+'_projecting_experiments'
    path = mkdir(connectivity_path / foldername)
    for area, experiments in tqdm(filtered_area_experiment_id_dict_2.items()):
        area_experiment_unionized_data[area] = {}
        for e in experiments:
            filename = 'area_id_'+str(area)+'exp_id_'+str(e)+'.csv'
            if os.path.isfile(connectivity_path / foldername / filename):
                temp_data = pd.read_csv(connectivity_path / foldername / filename)
            else:
                temp_data = RMAUnionizedData(experiment_id=e).data
                temp_data.to_csv(connectivity_path / foldername / filename)

            area_experiment_unionized_data[area][e] = temp_data

100%|██████████| 316/316 [00:12<00:00, 25.53it/s]


### Quality check for experiments with zero-valued projection metric in unionzed data

In [29]:
temp_dict = {}
for area in area_experiment_unionized_data:
    temp_dict[area] = {}
    for exp, exp_df in area_experiment_unionized_data[area].items():
        if (exp_df[exp_df['structure_id']==area][projection_metric] == 0).any():
            continue
        else:
            temp_dict[area][exp] = exp_df

del area_experiment_unionized_data
area_experiment_unionized_data = temp_dict

### Apply injection volume thresholding to experiments

In [30]:
def get_vol_from_downloaded_unionized_data(path, area, experiment):
    # For provided area and experiment id, reads corresponding csv file and returns volume
    filename = 'area_id_'+str(area)+'exp_id_'+str(experiment)+'.csv'
    if os.path.isfile(path / filename):
        temp_data = pd.read_csv(path / filename)
    else: return 0
    temp_data = temp_data[(temp_data['structure_id']==area) & (temp_data['hemisphere_id']==3)].reset_index(drop=True)
    return temp_data['volume'].item()

In [31]:
print(f'number of experiments BEFORE injection volume thresholding = {sum(len(area_experiment_unionized_data[area]) for area in area_experiment_unionized_data.keys())}')

foldername = f'unionized_data_from_hem_{HEMISPHERE_TO_FILTER}_{main_structure}_projecting_experiments'
folderpath = connectivity_path / foldername

temp_dict = {}

for area in tqdm(area_experiment_unionized_data):
    temp_dict[area] = {}
    for exp, exp_df in area_experiment_unionized_data[area].items():
        temp_vol = get_vol_from_downloaded_unionized_data(folderpath, area, exp)
        if temp_vol >= INJECTION_VOLUME_THRESHOLD:
            temp_dict[area][exp] = area_experiment_unionized_data[area][exp]
del area_experiment_unionized_data
area_experiment_unionized_data = temp_dict

print(f'number of experiments AFTER injection volume thresholding = {sum(len(area_experiment_unionized_data[area]) for area in area_experiment_unionized_data.keys())}')

number of experiments BEFORE injection volume thresholding = 1450


100%|██████████| 316/316 [00:13<00:00, 23.41it/s]

number of experiments AFTER injection volume thresholding = 897


### Separate experiments by the difference in ipsilateral and contralateral projections to target area

In [32]:
# Display projection information about experiments and hemisphere of target structure
# And collect experiments into two dictionaries

ipsilateral_projecting_exps = []
contralateral_projecting_exps = []
ipsilateral_dict = {}
contralateral_dict = {}

hem_ids = [2,1] # for getting the index of contralateral hemisphere to the one specified before

for area in area_experiment_unionized_data:
    ipsilateral_dict[area] = {}
    contralateral_dict[area] = {}
    for exp, exp_df in area_experiment_unionized_data[area].items():
        # Checking if unionized data of experiment has higher projection metric value in previosly selected hemisphere
        if exp_df[(exp_df['hemisphere_id']==HEMISPHERE_TO_FILTER) & (exp_df['structure_id']==main_structure_id)][projection_metric].item() > exp_df[(exp_df['hemisphere_id']==hem_ids[HEMISPHERE_TO_FILTER-1]) & (exp_df['structure_id']==main_structure_id)][projection_metric].item():
            ipsilateral_projecting_exps.append(exp)
            ipsilateral_dict[area][exp] = exp_df[exp_df['structure_id']==area]
        else: 
            contralateral_projecting_exps.append(exp)
            contralateral_dict[area][exp] = exp_df[exp_df['structure_id']==area]

print(len(ipsilateral_projecting_exps),'ipsilaterally projecting experiments')
print(len(contralateral_projecting_exps),'contralaterally projecting experiments')

466 ipsilaterally projecting experiments
431 contralaterally projecting experiments


### Display mouse lines of ipsilateral and contralateral experiments

In [33]:
print("Ipsilateral mouse lines")
ipsilateral_lines = [df[df['id']==x]['transgenic-line'].item() for x in ipsilateral_projecting_exps]
print(ipsilateral_lines)

print("Contralateral mouse lines")
contralateral_lines = [df[df['id']==x]['transgenic-line'].item() for x in contralateral_projecting_exps]
print(contralateral_lines)

Ipsilateral mouse lines
['Dbh-Cre_KH212', nan, nan, 'Drd3-Cre_KI196', 'Emx1-IRES-Cre', 'Ai75(RCL-nT)', 'Cux2-IRES-Cre', 'Ai75(RCL-nT)', 'Ai75(RCL-nT)', 'Ai75(RCL-nT)', 'Calb1-T2A-dgCre', 'Pcdh9-Cre_NP276', 'Rorb-IRES2-Cre', 'Rorb-IRES2-Cre', 'Gad2-IRES-Cre', nan, 'Cux2-IRES-Cre', 'Rorb-IRES2-Cre', 'A930038C07Rik-Tg1-Cre', nan, 'Tlx3-Cre_PL56', 'Slc18a2-Cre_OZ14', 'Oxtr-Cre_ON66', 'Gnb4-IRES2-CreERT2', 'Calb1-T2A-dgCre', 'Htr2a-Cre_KM207', 'Sim1-Cre_KJ18', nan, 'Grp-Cre_KH288', 'Grm2-Cre_MR90', nan, 'Slc32a1-IRES-Cre', 'Grm2-Cre_MR90', 'Syt17-Cre_NO14', 'Ppp1r17-Cre_NL146', nan, 'Emx1-IRES-Cre', 'Emx1-IRES-Cre', 'Emx1-IRES-Cre', 'Ai75(RCL-nT)', 'Ai75(RCL-nT)', 'Ai75(RCL-nT)', 'Ai75(RCL-nT)', 'Ai75(RCL-nT)', 'Ai75(RCL-nT)', 'Ai75(RCL-nT)', 'Ai75(RCL-nT)', 'Ai75(RCL-nT)', 'Ai75(RCL-nT)', 'Ai75(RCL-nT)', 'Ai75(RCL-nT)', 'Ai75(RCL-nT)', 'Ai75(RCL-nT)', 'Ai75(RCL-nT)', 'Ai75(RCL-nT)', 'Ai75(RCL-nT)', 'Ai75(RCL-nT)', 'Ai75(RCL-nT)', 'Trib2-F2A-CreERT2', 'Ai75(RCL-nT)', 'Ai75(RCL-nT)', 'Ai75(R

### For every brain area compute average projection metric and use it to compute weighted centroid

In [34]:
def xyz_weighted_centroid(coordinates):
    """
    Returns xyz coordinates of a centroid weighted by vertices and the associated average projection metric. Computed to determine central coordinate within a brain region, weighted by projection metric of each experiment injected in that region.
    
    cx = (v1x*m1 + v2x*m2 + ... vnx*mn) / (m1 + m2 .... mn) 
    cy = (v1y*m1 + v2y*m2 + ... vny*mn) / (m1 + m2 .... mn)
    cz = (v1z*m1 + v2z*m2 + ... vnz*mn) / (m1 + m2 .... mn)
    
    where v1x, v1y and v1z are xyz coordinates of vertex 1 and m1 is its weight.
    
        Parameters:
            coordinates (List): list of nested lists containing coordinates and weight of each vertex nested [[x1, y1, z3, w1], [x2, y2, z2, w2], ...].
        
        Returns:
            centroid_point (List): location of centroid and related average value of projection metric in the form of [x, y, z, avg_projection].
    """
    denom = sum(exp[3] for exp in coordinates)
    centroid_point = [int(sum(exp[0]*exp[3] for exp in coordinates) / denom), int(sum(exp[1]*exp[3] for exp in coordinates) / denom), int(sum(exp[2]*exp[3] for exp in coordinates) / denom), denom / len(coordinates)]
    
    return centroid_point

In [35]:
hem_ids = [2,1] # for getting the index of contralateral hemisphere to the one specified before
ipsilateral_centroids_dict = {}
contralateral_centroids_dict = {}

for area in ipsilateral_dict:
    coordinates = [[exp[exp['hemisphere_id']==HEMISPHERE_TO_FILTER]['max_voxel_x'].item(), exp[exp['hemisphere_id']==HEMISPHERE_TO_FILTER]['max_voxel_y'].item(), exp[exp['hemisphere_id']==HEMISPHERE_TO_FILTER]['max_voxel_z'].item(), exp[exp['hemisphere_id']==HEMISPHERE_TO_FILTER][projection_metric].item()] for exp in ipsilateral_dict[area].values()]
    if len(coordinates) == 0: centroid_xyz = None
    else: centroid_xyz = xyz_weighted_centroid(coordinates)
    if centroid_xyz: ipsilateral_centroids_dict[area] = centroid_xyz
print(len(ipsilateral_centroids_dict.keys()),'ipsilateral centroids computed out of',str(len(ipsilateral_dict.keys())),'regions')

for area in contralateral_dict:
    coordinates = [[exp[exp['hemisphere_id']==hem_ids[HEMISPHERE_TO_FILTER-1]]['max_voxel_x'].item(), exp[exp['hemisphere_id']==hem_ids[HEMISPHERE_TO_FILTER-1]]['max_voxel_y'].item(), exp[exp['hemisphere_id']==hem_ids[HEMISPHERE_TO_FILTER-1]]['max_voxel_z'].item(), exp[exp['hemisphere_id']==hem_ids[HEMISPHERE_TO_FILTER-1]][projection_metric].item()] for exp in contralateral_dict[area].values()]
    if len(coordinates) == 0: centroid_xyz = None
    else: centroid_xyz = xyz_weighted_centroid(coordinates)
    if centroid_xyz: contralateral_centroids_dict[area] = centroid_xyz
print(len(contralateral_centroids_dict.keys()),'contralateral centroids computed out of',str(len(contralateral_dict.keys())),'regions')

61 ipsilateral centroids computed out of 316 regions
66 contralateral centroids computed out of 316 regions


### Saving computed centroids

In [36]:
ipsilateral_centroids_dict[795]

[9880, 3635, 5190, 0.8192550033919425]

In [37]:
foldername = f'centroids_{projection_metric}_hem_id_{HEMISPHERE_TO_FILTER}_inj_vol_thresh_{INJECTION_VOLUME_THRESHOLD}_{main_structure}'
folderpath = connectivity_path / foldername
mkdir(folderpath)

PosixPath('/home/ikharitonov/Desktop/data/connectivity/centroids_normalized_projection_volume_hem_id_1_inj_vol_thresh_1.5_VISpm')

In [38]:
filename = f'ipsilateral_centroids_dict_hem_{HEMISPHERE_TO_FILTER}_inj_vol_thresh_{INJECTION_VOLUME_THRESHOLD}_{main_structure}.pkl'
with open(folderpath / filename, 'wb') as f: pickle.dump(ipsilateral_centroids_dict, f)
filename = f'contralateral_centroids_dict_hem_{HEMISPHERE_TO_FILTER}_inj_vol_thresh_{INJECTION_VOLUME_THRESHOLD}_{main_structure}.pkl'
with open(folderpath / filename, 'wb') as f: pickle.dump(contralateral_centroids_dict, f)

In [39]:
# Reading from a file
filename = f'ipsilateral_centroids_dict_hem_{HEMISPHERE_TO_FILTER}_inj_vol_thresh_{INJECTION_VOLUME_THRESHOLD}_{main_structure}.pkl'
with open(folderpath / filename, 'rb') as f: ipsilateral_centroids_dict = pickle.load(f)
filename = f'contralateral_centroids_dict_hem_{HEMISPHERE_TO_FILTER}_inj_vol_thresh_{INJECTION_VOLUME_THRESHOLD}_{main_structure}.pkl'
with open(folderpath / filename, 'rb') as f: contralateral_centroids_dict = pickle.load(f)